In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf

C:\Soft\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 32
num_batches = 100

In [3]:
def print_activations(t):
    print(t.op.name, '', t.get_shape().as_list())

In [4]:
def inference(images):
    parameters = []

    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(
            tf.truncated_normal([11, 11, 3, 64], dtype=tf.float32,
                                stddev=1e-1),
            name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
        biases = tf.Variable(
            tf.constant(0.0, shape=[64], dtype=tf.float32),
            trainable=True,
            name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv1)

    lrn1 = tf.nn.lrn(
        conv1, 4, bias=1.0, alpha=0.001 / 9, beta=0.75, name='lrn1')
    pool1 = tf.nn.max_pool(
        lrn1,
        ksize=[1, 3, 3, 1],
        strides=[1, 2, 2, 1],
        padding='VALID',
        name='pool1')
    print_activations(pool1)

    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(
            tf.truncated_normal([5, 5, 64, 192], dtype=tf.float32,
                                stddev=1e-1),
            name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(
            tf.constant(0.0, shape=[192], dtype=tf.float32),
            trainable=True,
            name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv2)

    lrn2 = tf.nn.lrn(
        conv2, 4, bias=1.0, alpha=0.001 / 9, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(
        lrn2,
        ksize=[1, 3, 3, 1],
        strides=[1, 2, 2, 1],
        padding="VALID",
        name='pool2')
    print_activations(pool2)

    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(
            tf.truncated_normal([3, 3, 192, 384],
                                dtype=tf.float32,
                                stddev=1e-1),
            name='weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(
            tf.constant(0.0, shape=[384], dtype=tf.float32),
            trainable=True,
            name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv3)

    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(
            tf.truncated_normal([3, 3, 384, 256],
                                dtype=tf.float32,
                                stddev=1e-1),
            name='weights')
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(
            tf.constant(0.0, shape=[256], dtype=tf.float32),
            trainable=True,
            name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv4)

    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(
            tf.truncated_normal([3, 3, 256, 256],
                                dtype=tf.float32,
                                stddev=1e-1),
            name='weights')
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(
            tf.constant(0.0, shape=[256], dtype=tf.float32),
            trainable=True,
            name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv5)

    pool5 = tf.nn.max_pool(
        conv5,
        ksize=[1, 3, 3, 1],
        strides=[1, 2, 2, 1],
        padding='VALID',
        name='pool5')
    print_activations(pool5)

    return pool5, parameters

In [5]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i > num_steps_burn_in:
            if not i % 10:
                print('%s: step %d,duration = %.3f' %
                      (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
          (datetime.now(), info_string, num_batches, mn, sd))

In [6]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(
            tf.random_normal([batch_size, image_size, image_size, 3],
                             dtype=tf.float32,
                             stddev=1e-1))
        pool5, parameters = inference(images)

        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)

        time_tensorflow_run(sess, pool5, "Forward")

        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective, parameters)
        time_tensorflow_run(sess, grad, "Forward-backward")

In [7]:
run_benchmark()

conv1  [32, 56, 56, 64]
pool1  [32, 27, 27, 64]
conv2  [32, 27, 27, 192]
pool2  [32, 13, 13, 192]
conv3  [32, 13, 13, 384]
conv4  [32, 13, 13, 256]
conv5  [32, 13, 13, 256]
pool5  [32, 6, 6, 256]
2019-02-27 17:11:48.504065: step 10,duration = 0.756
2019-02-27 17:11:55.879688: step 20,duration = 0.671
2019-02-27 17:12:03.184324: step 30,duration = 0.704
2019-02-27 17:12:10.086665: step 40,duration = 0.660
2019-02-27 17:12:17.494442: step 50,duration = 0.760
2019-02-27 17:12:24.953077: step 60,duration = 0.759
2019-02-27 17:12:32.534296: step 70,duration = 0.751
2019-02-27 17:12:40.131859: step 80,duration = 0.766
2019-02-27 17:12:47.738350: step 90,duration = 0.759
2019-02-27 17:12:53.139595: Forward across 100 steps, 0.721 +/- 0.093 sec / batch
2019-02-27 17:14:00.729844: step 10,duration = 3.145
2019-02-27 17:14:34.144066: step 20,duration = 3.379
2019-02-27 17:15:07.576090: step 30,duration = 3.363
2019-02-27 17:15:41.209521: step 40,duration = 3.372
2019-02-27 17:16:14.752386: step 